#### Метод Галеркина

In [1]:
import numpy as np

xbeg, xend = 0, 1

def Galerkin(N):
    def yfunc(x):
        return (181/230) * x + (119/2300) * x**2 + (63/2300) * x**3 + (77/575) * x**4

    xvals = np.linspace(xbeg, xend, num=N)
    yvals = [yfunc(x) for x in xvals]
    return yvals

#### Разностная схема - прогонка

In [2]:
def epsilon_ip1(xi, epsiloni, h):
    Ai = 1 + xi * h
    Bi = 2 * (1 - h**2)
    Di = (1 - xi * h)
    Fi = - xi * h**2
    return Di / (Bi - Ai * epsiloni)

def eta_ip1(xi, epsiloni, etai, h):
    Ai = 1 + xi * h
    Bi = 2 * (1 - h**2)
    Di = (1 - xi * h)
    Fi = - xi * h**2
    return (Ai * etai + Fi) / (Bi - Ai * epsiloni)

def progonka(N):
    N -= 1
    h = (xend - xbeg) / N
    epsilon1, eta1 = 0, 0
    xvals = [xbeg]
    epsilonVals, etaVals = [epsilon1], [eta1]

    for i in range(N - 1):
        epsilonNext = epsilon_ip1(xvals[-1], epsilonVals[-1], h)
        etaNext = eta_ip1(xvals[-1], epsilonVals[-1], etaVals[-1], h)
        epsilonVals.append(epsilonNext)
        etaVals.append(etaNext)
        xvals.append(xvals[-1] + h)

    print(len(epsilonVals) == N)
    print(f'epsilonVals={epsilonVals}')
    print(f'etaVals={etaVals}')

    # yN = (2*h + epsilonVals[-2]*etaVals[-1] + etaVals[-2]) / (1 - epsilonVals[-2]*epsilonVals[-1])
    yN = (h + etaVals[-1]) / (1 - epsilonVals[-1])
    print(f'yN = {yN}')
    yvals = [0] * N 
    yvals[N - 1] = yN 
    for i in range(N - 2, -1, -1):
        yi = epsilonVals[i + 1] * yvals[i + 1] + etaVals[i + 1]
        yvals[i] = yi
    
    return [0] + yvals

# progonka()

In [6]:
import plotly.express as px
import pandas as pd

cntSteps = 6
xvals = np.linspace(xbeg, xend, num=cntSteps)
galerkin_yvals = Galerkin(cntSteps)
progonka_yvals = progonka(cntSteps)

df1 = pd.DataFrame({"x": xvals, "y": galerkin_yvals, "label": ["Галеркин3 y(x)"] * len(galerkin_yvals) })
df2 = pd.DataFrame({"x": xvals, "y": progonka_yvals, "label": ["Прогонка y(x)"] * len(progonka_yvals) })
df = pd.concat([df1, df2])
fig1 = px.line(df, x="x", y="y", color="label", title="", markers = True)
fig1.show()


True
epsilonVals=[0, 0.5208333333333334, 0.6964933494558646, 0.7878147779986747, 0.8480722437588215]
etaVals=[0, 0.0, -0.00580411124546554, -0.019068919814446663, -0.04371156142827039]
yN = 1.0287023414183039
